In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum as spark_sum
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as F
import networkx as nx
import numpy as np
import json
import time
import math
from gensim.models import Word2Vec
from scipy.io import loadmat

# Initialize a Spark session
conf = SparkConf().setAppName("Node2VecPySpark")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Read the file content
# file_path = '/Users/tesfaasmara/Documents/GitHub/cs183/nbs/blogcatalog.mat'
# mat_content_rdd = sc.wholeTextFiles(file_path)
# mat_content = mat_content_rdd.collect()[0][1]

# # Load the MATLAB file into a Python object
# mat_data = loadmat(file_name=file_path)

# Define a list of MAT file paths or load from HDFS, etc.
mat_files = ["/Users/tesfaasmara/Documents/GitHub/cs183/nbs/blogcatalog.mat"]

# Create an RDD of MAT file data
mat_rdd = sc.parallelize(mat_files)

# Load and parse the MAT files using pyspark-matplotlib
parsed_data_rdd = mat_rdd.map(lambda file_path: loadmat(file_path))

# Extract the adjacency matrix from the object
adj_matrix = parsed_data_rdd.collect()[0]['network']

group_labels = parsed_data_rdd.collect()[0]['group']
# Get the index of the maximum value for each row
labels = group_labels.argmax(axis=1).A1

# Convert the adjacency matrix to a NetworkX graph
G = nx.from_numpy_array(adj_matrix)

nx.set_edge_attributes(G, nx.get_edge_attributes(G, "weight"), "capacity")

# Print some basic information about the graph
print(G)  

def jaccard_coefficient(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = len(curr)+len(dest)
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def speed_up(G, num_workers, transition_matrix_function):
    nodes = G.nodes
    # Split the nodes into chunks for each worker
    node_chunks = np.array_split(nodes, num_workers)

    # Use joblib to parallelize the calculation of ss_weight
    ss_weights = Parallel(n_jobs=num_workers)(
        delayed(transition_matrix_function)(G, current_node, destination)
        for chunk in node_chunks
        for current_node in chunk
        for destination in nodes
    )

    # Reshape the ss_weights list into a matrix
    ss_weights_matrix1 = np.reshape(ss_weights, (len(nodes), len(nodes)))
    # ss_weights_matrix1 = ss_weights_matrix1/ss_weights_matrix1.sum(axis=1, keepdims=True)


    # check if row sums are zero
    row_sums = ss_weights_matrix1.sum(axis=1)
    zero_rows = np.where(row_sums == 0)[0]

    # set all elements in zero rows to zero, except for diagonal element
    ss_weights_matrix1[zero_rows, :] = 0
    for i in zero_rows:
        ss_weights_matrix1[i, i] = 1

    row_sums = ss_weights_matrix1.sum(axis=1)
    # normalize matrix by row sums
    ss_weights_matrix1 = np.divide(ss_weights_matrix1, row_sums[:, np.newaxis])
    return ss_weights_matrix1

# Example usage

# Call the jaccard_coefficient function
result = jaccard_coefficient(G, 1, 2)
print("Jaccard coefficient:", result)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/23 16:04:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Graph with 10312 nodes and 333983 edges
Jaccard coefficient: 0.95


23/10/24 01:05:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 910075 ms exceeds timeout 120000 ms
23/10/24 01:05:03 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/24 01:20:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

In [1]:
# Extract the adjacency matrix from the object
adj_matrix = mat_data['network']

group_labels = mat_data['group']

# Get the index of the maximum value for each row
labels = group_labels.argmax(axis=1).A1

# Convert the adjacency matrix to a NetworkX graph
G = nx.from_numpy_array(adj_matrix)



def jaccard_coefficient(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = len(curr)+len(dest)
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def speed_up(G, num_workers, transition_matrix_function):
    nodes = G.nodes
    # Split the nodes into chunks for each worker
    node_chunks = np.array_split(nodes, num_workers)

    # Use joblib to parallelize the calculation of ss_weight
    ss_weights = Parallel(n_jobs=num_workers)(
        delayed(transition_matrix_function)(G, current_node, destination)
        for chunk in node_chunks
        for current_node in chunk
        for destination in nodes
    )

    # Reshape the ss_weights list into a matrix
    ss_weights_matrix1 = np.reshape(ss_weights, (len(nodes), len(nodes)))
    # ss_weights_matrix1 = ss_weights_matrix1/ss_weights_matrix1.sum(axis=1, keepdims=True)


    # check if row sums are zero
    row_sums = ss_weights_matrix1.sum(axis=1)
    zero_rows = np.where(row_sums == 0)[0]

    # set all elements in zero rows to zero, except for diagonal element
    ss_weights_matrix1[zero_rows, :] = 0
    for i in zero_rows:
        ss_weights_matrix1[i, i] = 1

    row_sums = ss_weights_matrix1.sum(axis=1)
    # normalize matrix by row sums
    ss_weights_matrix1 = np.divide(ss_weights_matrix1, row_sums[:, np.newaxis])
    return ss_weights_matrix1

# Example usage

# Call the jaccard_coefficient function
result = jaccard_coefficient(G, 1, 2)
print("Jaccard coefficient:", result)




NameError: name 'mat_data' is not defined

Differences between PySpark and having Spark installed in your machine "spark-submit"